<a href="https://colab.research.google.com/github/SavchukDI/3d_ml_mai/blob/main/3dml_lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SimSwap
Reference: https://github.com/neuralchen/SimSwap

## Prepare code

### SimSwap

In [ ]:
%cd /content
!git clone https://github.com/woctezuma/SimSwap.git
%cd /content/SimSwap/
!git checkout no-logo

/content
Cloning into 'SimSwap'...
remote: Enumerating objects: 476, done.
remote: Total 476 (delta 0), reused 0 (delta 0), pack-reused 476
Receiving objects: 100% (476/476), 121.02 MiB | 28.29 MiB/s, done.
Resolving deltas: 100% (190/190), done.
/content/SimSwap
Branch 'no-logo' set up to track remote branch 'no-logo' from 'origin'.
Switched to a new branch 'no-logo'


In [ ]:
!pip install insightface==0.2.1 onnxruntime moviepy > /dev/null
!pip install googledrivedownloader > /dev/null
!pip install imageio==2.4.1 > /dev/null

### (optional) GPEN

In [ ]:
%cd /content
!git clone https://github.com/yangxy/GPEN.git

# Remove the example input
%rm -rf /content/GPEN/examples
%mkdir -p /content/GPEN/examples/imgs

/content
Cloning into 'GPEN'...
remote: Enumerating objects: 392, done.
remote: Counting objects: 100% (392/392), done.
remote: Compressing objects: 100% (322/322), done.
remote: Total 392 (delta 113), reused 331 (delta 57), pack-reused 0
Receiving objects: 100% (392/392), 52.88 MiB | 21.49 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [ ]:
%pip install ninja

     |████████████████████████████████| 108 kB 5.5 MB/s 


## Prepare models

### SimSwap

In [ ]:
%cd /content/SimSwap

/content/SimSwap


In [ ]:
from google_drive_downloader import GoogleDriveDownloader

GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
                                    dest_path='./arcface_model/arcface_checkpoint.tar')

GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
                                    dest_path='./checkpoints.zip')

In [ ]:
!wget --no-check-certificate \
 https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w \
 -O antelope.zip

--2021-12-25 02:18:04--  https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w
Resolving sh23tw.dm.files.1drv.com (sh23tw.dm.files.1drv.com)... 13.107.42.12
Connecting to sh23tw.dm.files.1drv.com (sh23tw.dm.files.1drv.com)|13.107.42.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248024513 (237M) [application/zip]
Saving to: ‘antelope.zip’

antelope.zip        100%[===================>] 236.53M  46.5MB/s    in 5.3s    

2021-12-25 02:18:09 (45.0 MB/s) - ‘antelope.zip’ saved [248024513/248024513]



In [ ]:
!unzip ./checkpoints.zip  -d ./checkpoints

!unzip antelope.zip -d ./insightface_func/models/

Archive:  ./checkpoints.zip
   creating: ./checkpoints/people/
  inflating: ./checkpoints/people/iter.txt  
  inflating: ./checkpoints/people/latest_net_D1.pth  
  inflating: ./checkpoints/people/latest_net_D2.pth  
  inflating: ./checkpoints/people/latest_net_G.pth  
  inflating: ./checkpoints/people/loss_log.txt  
  inflating: ./checkpoints/people/opt.txt  
   creating: ./checkpoints/people/web/
   creating: ./checkpoints/people/web/images/
Archive:  antelope.zip
   creating: ./insightface_func/models/antelope/
  inflating: ./insightface_func/models/antelope/glintr100.onnx  
  inflating: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx  


### (optional) GPEN

In [ ]:
%cd /content/GPEN/weights
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth
!wget https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-512.pth

/content/GPEN/weights
--2021-12-25 02:18:16--  https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/RetinaFace-R50.pth
Resolving public-vigen-video.oss-cn-shanghai.aliyuncs.com (public-vigen-video.oss-cn-shanghai.aliyuncs.com)... 47.101.88.25
Connecting to public-vigen-video.oss-cn-shanghai.aliyuncs.com (public-vigen-video.oss-cn-shanghai.aliyuncs.com)|47.101.88.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109497761 (104M) [application/octet-stream]
Saving to: ‘RetinaFace-R50.pth’

RetinaFace-R50.pth  100%[===================>] 104.42M  10.0MB/s    in 19s     

2021-12-25 02:18:37 (5.36 MB/s) - ‘RetinaFace-R50.pth’ saved [109497761/109497761]

--2021-12-25 02:18:37--  https://public-vigen-video.oss-cn-shanghai.aliyuncs.com/robin/models/GPEN-512.pth
Resolving public-vigen-video.oss-cn-shanghai.aliyuncs.com (public-vigen-video.oss-cn-shanghai.aliyuncs.com)... 47.101.88.25
Connecting to public-vigen-video.oss-cn-shanghai.aliyuncs.com (public

## Prepare data

### Download

In [ ]:
%cd /content

!wget https://www.themoviedb.org/t/p/w600_and_h900_bestv2/ufxkgL7bALTQ1QKMRY3i3j1q25R.jpg -O adam_savage.png
!wget https://icdn4.digitaltrends.com/image/digitaltrends/obi-wan-kenobi-1300x650.jpg -O obi_wan.png

/content
--2021-12-25 02:19:18--  https://www.themoviedb.org/t/p/w600_and_h900_bestv2/ufxkgL7bALTQ1QKMRY3i3j1q25R.jpg
Resolving www.themoviedb.org (www.themoviedb.org)... 65.8.56.90, 65.8.56.112, 65.8.56.27, ...
Connecting to www.themoviedb.org (www.themoviedb.org)|65.8.56.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76346 (75K) [image/jpeg]
Saving to: ‘adam_savage.png’

adam_savage.png     100%[===================>]  74.56K  --.-KB/s    in 0.01s   

2021-12-25 02:19:19 (5.20 MB/s) - ‘adam_savage.png’ saved [76346/76346]

--2021-12-25 02:19:19--  https://icdn4.digitaltrends.com/image/digitaltrends/obi-wan-kenobi-1300x650.jpg
Resolving icdn4.digitaltrends.com (icdn4.digitaltrends.com)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to icdn4.digitaltrends.com (icdn4.digitaltrends.com)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94316 (92K) [image/jpeg]
Saving to: ‘obi_wan.png’

obi_wan.png         1

In [ ]:
input_fname = '/content/adam_savage.png'
output_fname = '/content/obi_wan.png'

### Convert to JPG

Images should not be too large, hence the (arbitrary) limitation of 1024 length.

In [ ]:
def get_new_size(img_size,
                 max_allowed_length = 1024):

  new_img_size = []
  if any(max_allowed_length < sz for sz in img_size):
    long_length = max(img_size)
    ratio = max_allowed_length / long_length
  else:
    ratio = 1.0

    new_img_size = [
                    int(ratio*sz)
                    for sz in img_size
                    ]

  return tuple(new_img_size)

In [ ]:
from PIL import Image

allow_resize = False

for fname in [input_fname, output_fname]:
  jpg_fname = fname.replace('.png', '.jpg')

  try:
    img = Image.open(fname)
  except FileNotFoundError:
    continue

  new_size = get_new_size(img.size, max_allowed_length = 1024)
  if allow_resize:
    print(f'Resizing from {img.size} to {new_size}')
    img.resize(new_size)

  print(f'Saving to {jpg_fname}')
  img.convert('RGB').save(jpg_fname)

jpg_input = input_fname.replace('.png', '.jpg')
jpg_output = output_fname.replace('.png', '.jpg')

Saving to /content/adam_savage.jpg
Saving to /content/obi_wan.jpg


## Run SimSwap

### Single

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/single/

!python test_wholeimage_swapsingle.py \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/single/ > /dev/null


/content/SimSwap
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. Saved a reverse patch to DataParallel.patch. Run `patch -p0 < DataParallel.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. Saved a reverse patch to Conv2d.patch. Run `patch -p0 < Conv2d.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. Saved a reverse patch to BatchNorm2d.patch. Run `patch -p0 < BatchNorm2d.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: S

### Multi

In [ ]:
%cd /content/SimSwap
%mkdir -p /content/output/multi/

!python test_wholeimage_swapmutil.py \
 --isTrain false  --name people \
 --Arc_path arcface_model/arcface_checkpoint.tar \
 --pic_a_path {jpg_input} \
 --pic_b_path {jpg_output} \
 --output_path /content/output/multi/ > /dev/null


/content/SimSwap
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. Saved a reverse patch to DataParallel.patch. Run `patch -p0 < DataParallel.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. Tried to save a patch, but couldn't create a writable file Conv2d.patch. Make sure it doesn't exist and your working directory is writable.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. Saved a reverse patch to BatchNorm2d.patch. Run `patch -p0 < BatchNorm2d.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3

## (optional) Post-process with GPEN

In [ ]:
%cp /content/output/single/result_whole_swapsingle.jpg /content/GPEN/examples/imgs/
%cp /content/output/multi/result_whole_swapmutil.jpg /content/GPEN/examples/imgs/

In [ ]:
%cd /content/GPEN
!python face_enhancement.py

/content/GPEN
Traceback (most recent call last):
  File "face_enhancement.py", line 124, in <module>
    faceenhancer = FaceEnhancement(size=args.size, model=args.model, use_sr=args.use_sr, sr_model=args.sr_model, channel_multiplier=args.channel_multiplier, narrow=args.narrow, key=args.key, device='cuda' if args.use_cuda else 'cpu')
  File "face_enhancement.py", line 23, in __init__
    self.srmodel =  RealESRNet(base_dir, sr_model, device=device)
  File "/content/GPEN/sr_model/real_esrnet.py", line 12, in __init__
    self.load_srmodel(base_dir, model)
  File "/content/GPEN/sr_model/real_esrnet.py", line 19, in load_srmodel
    loadnet = torch.load(os.path.join(self.base_dir, 'weights', model+'.pth'))
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 594, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  F